In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Custom Dataset
class WifiCSIDataset(Dataset):
    def __init__(self, data_dir):
        self.data_paths = []
        self.labels = []
        self.label_map = {}
        label_id = 0

        for label_name in sorted(os.listdir(data_dir)):
            label_path = os.path.join(data_dir, label_name)
            if os.path.isdir(label_path):
                if label_name not in self.label_map:
                    self.label_map[label_name] = label_id
                    label_id += 1

                for file in os.listdir(label_path):
                    if file.endswith(".npy"):
                        self.data_paths.append(os.path.join(label_path, file))
                        self.labels.append(self.label_map[label_name])

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        data = np.load(self.data_paths[idx]).astype(np.float32)  # drops imaginary part
        label = self.labels[idx]
        tensor = torch.from_numpy(data)
        return tensor, label

# Collate function to pad sequences
def pad_collate_fn(batch):
    inputs, labels = zip(*batch)
    lengths = [x.shape[0] for x in inputs]
    max_len = max(lengths)

    padded_inputs = torch.stack([torch.cat([x, torch.zeros(max_len - x.shape[0], x.shape[1])]) for x in inputs])
    labels = torch.tensor(labels)
    return padded_inputs, labels


In [2]:
# ----- Dataset Path -----
data_path = r"C:\Users\harsh\Downloads\Room2_npy"  
# ----- Create Dataset -----
dataset = WifiCSIDataset(data_path)
print(f"Loaded {len(dataset)} samples from:", list(dataset.label_map.keys()))

# ----- Train/Validation Split -----
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# ----- Dataloaders -----
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn)


Loaded 114 samples from: ['kneel', 'liedown', 'pickup', 'sit', 'sitrotate', 'stand', 'standrotate', 'walk']


In [5]:
class WiSigProTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, hidden_dim, num_classes, num_layers=2):
        super(WiSigProTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        x = self.embedding(x)                     # → (batch_size, seq_len, hidden_dim)
        x = x.permute(1, 0, 2)                    # → (seq_len, batch_size, hidden_dim)
        x = self.transformer_encoder(x)          # → (seq_len, batch_size, hidden_dim)
        x = x.permute(1, 2, 0)                    # → (batch_size, hidden_dim, seq_len)
        x = self.pooling(x).squeeze(2)            # → (batch_size, hidden_dim)
        return self.classifier(x)                 # → (batch_size, num_classes)


In [7]:
# ----- Model, Loss, Optimizer -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sample_input, _ = dataset[0]
input_dim = sample_input.shape[1]
num_classes = len(dataset.label_map)
hidden_dim = 120
num_heads = 5  # must divide hidden_dim
num_epochs = 10

model = WiSigProTransformer(input_dim, num_heads, hidden_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# ----- Training Loop -----
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/total:.4f}, Accuracy: {acc:.2f}%")

    log_file = "training_log.csv"
    if not Path(log_file).exists():
        with open(log_file, mode='w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Epoch', 'Loss', 'Accuracy'])

    with open(log_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([epoch + 1, running_loss / total, acc])






C:\Users\harsh\AppData\Local\Temp\ipykernel_13932\1051092239.py:33: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.load(self.data_paths[idx]).astype(np.float32)  # drops imaginary part
C:\Users\harsh\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
C:\Users\harsh\AppData\Local\Temp\ipykernel_13932\1051092239.py:33: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.load(self.data_paths[idx]).astype(np.float32)  # drops imaginary part


RuntimeError: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 16000000000 bytes.

In [ ]:
# ----- Evaluation -----
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ----- Confusion Matrix -----
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(dataset.label_map.keys()))
disp.plot(xticks_rotation=45)
plt.title("Validation Confusion Matrix")
plt.show()

# ----- Inference on One Sample -----
sample, label = dataset[0]
sample = sample.unsqueeze(0).to(device)  # add batch dimension
model.eval()
with torch.no_grad():
    output = model(sample)
    pred = torch.argmax(output, dim=1).item()

print(f"True label: {label}, Predicted label: {pred} ({list(dataset.label_map.keys())[pred]})")


In [ ]:
# ----- Save the model -----
torch.save(model.state_dict(), "transformer_wifi_csi.pth")
print("Model saved to transformer_wifi_csi.pth")

# ----- To load the model later -----
 model = WiSigProTransformer(input_dim, num_heads, hidden_dim, num_classes)
 model.load_state_dict(torch.load("transformer_wifi_csi.pth"))
 model.to(device)
 print(" Model loaded")
